In [ ]:
import os, random
from datasets import Dataset
import numpy as np
import pandas as pd
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          TrainingArguments, Trainer)

from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, accuracy_score

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

DATA_PATH = os.environ.get("DATA_DIR", "../data/synth_clinical")
NOTES_HARD_CSV = os.path.join(DATA_PATH, "notes_hard.csv")
LABELS_CSV = os.path.join(DATA_PATH, "docs_labels_hard.csv")

notes = pd.read_csv(NOTES_HARD_CSV)
labels = pd.read_csv(LABELS_CSV)
df = notes.merge(labels[['doc_id', 'hard_label']], on='doc_id', how='left')
df['label'] = df['hard_label'].astype(int)

assert os.path.exists(NOTES_HARD_CSV) and os.path.exists(LABELS_CSV), f"Missing notes.csv at {NOTES_HARD_CSV} or {LABELS_CSV}"
print("Using data:", NOTES_HARD_CSV, LABELS_CSV)
print(df['label'].value_counts())
print(df.head(2))

In [ ]:
ds=Dataset.from_pandas(df[['text','label']])
ds=ds.train_test_split(test_size=0.2, seed=SEED)
len(ds['train']), len(ds['test'])

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = logits - logits.max(axis=1, keepdims=True)
    probs = np.exp(probs); probs = probs[:,1] / probs.sum(axis=1)
    preds=(probs>=0.5).astype(int)
    return {
        'accuracy': accuracy_score(labels, preds),
        'f1': f1_score(labels, preds),
        'auroc': roc_auc_score(labels, probs),
        'auprc': average_precision_score(labels, probs)
    }


In [ ]:
def run_model(model_name:str, max_len=768, epochs=3, batch=16, lr=2e-5, fp16=True):
    tok=AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)
    enc=ds.map(lambda x: tok(x['text'], max_length=max_len, truncation=True), batched=True, remove_columns=['text'])
    model=AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
    args=TrainingArguments(
        output_dir=f"./reports/doc_cls_hard_{model_name.replace('/','_')}",
        per_device_train_batch_size=batch,
        per_device_eval_batch_size=batch,
        num_train_epochs=epochs,
        learning_rate=lr,
        fp16=fp16,
        logging_steps=50,
        evaluation_strategy='epoch',
        save_strategy='no',
        report_to=[],
        seed=SEED
    )
    trainer=Trainer(model=model, args=args, train_dataset=enc['train'], eval_dataset=enc['test'], tokenizer=tok, compute_metrics=compute_metrics)
    import time; t0=time.time(); trainer.train(); dur=time.time()-t0
    metrics=trainer.evaluate(); metrics['seconds']=dur; return metrics


In [ ]:
results={}
for name, mn in [("BioClinical ModernBERT", "thomas-sounack/BioClinical-ModernBERT-base"),
                 ("ModernBERT (vanilla)", "answerdotai/ModernBERT-base")]:
    print(f"\n==== Training {name}: {mn} ====")
    results[name]=run_model(mn)
results


In [ ]:
pd.DataFrame(results).T[['accuracy','f1','auroc','auprc','eval_loss','seconds']]
